### Bibliotecas de programación paralela en C 


### Pthreads

POSIX Threads, comúnmente conocidos como Pthreads, son un estándar de la biblioteca de hilos definido por POSIX (Portable Operating System Interface) que proporciona una API para crear y gestionar hilos a nivel de usuario. Pthreads permite a los desarrolladores escribir programas concurrentes y paralelos, mejorando el rendimiento de las aplicaciones al aprovechar los sistemas de multiprocesador y los núcleos múltiples en CPUs modernas.



### API de Pthreads

Pthreads proporciona varias funciones para gestionar hilos, algunas de las más importantes incluyen:

**pthread_create:**

Prototipo:`int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine) (void *), void *arg);`

Descripción: Crea un nuevo hilo que ejecuta la función especificada por start_routine.

**pthread_join:**

Prototipo: `int pthread_join(pthread_t thread, void **retval);`
Descripción: Espera a que el hilo especificado termine y, opcionalmente, obtiene el valor devuelto por el hilo.

**pthread_exit:**

Prototipo: `void pthread_exit(void *retval);`
Descripción: Termina la ejecución del hilo que llama a esta función y devuelve un valor opcional.

**pthread_mutex_lock** y **pthread_mutex_unlock:**

Descripción: Estas funciones se utilizan para bloquear y desbloquear un mutex, respectivamente, para asegurar la exclusión mutua en secciones críticas del código.


In [ ]:
// mutexes 
pthread_mutex_t lock;
pthread_mutex_init(&lock, NULL);

pthread_mutex_lock(&lock);
// Sección crítica
pthread_mutex_unlock(&lock);


In [ ]:
// variables de condicion
pthread_cond_t cond;
pthread_mutex_t lock;

pthread_mutex_lock(&lock);
pthread_cond_wait(&cond, &lock);
// Espera hasta que otra señal condicional despierte el hilo.
pthread_mutex_unlock(&lock);


In [ ]:
// Barrier

pthread_barrier_t barrier;
pthread_barrier_init(&barrier, NULL, N); // N es el número de hilos

pthread_barrier_wait(&barrier);
// Todos los hilos deben alcanzar este punto antes de continuar.


#### Ejemplo

In [ ]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

void* print_message(void* ptr) {
    char* message = (char*) ptr;
    printf("%s \n", message);
    return NULL;
}

int main() {
    pthread_t thread1, thread2;
    const char* message1 = "Hello from Thread 1";
    const char* message2 = "Hello from Thread 2";

    pthread_create(&thread1, NULL, print_message, (void*) message1);
    pthread_create(&thread2, NULL, print_message, (void*) message2);

    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);

    return 0;
}


Para compilar y ejecutar: 

gcc -o pthreads_example pthreads_example.c -lpthread

./pthreads_example


**Consideraciones y buenas prácticas**

- Evitación de Deadlocks: Los deadlocks ocurren cuando dos o más hilos se bloquean mutuamente esperando recursos que los otros tienen. Para evitarlo, siempre adquiere los bloqueos en el mismo orden y usa técnicas como el tiempo de espera para detectar y manejar deadlocks.
- Diseño de programas escalables: Diseña tus programas para que puedan escalar con el número de núcleos disponibles, evitando la sobrecarga excesiva de creación y destrucción de hilos.
- Uso adecuado de sincronización: Utiliza las primitivas de sincronización adecuadas según el caso, y evita la sincronización excesiva que puede llevar a una degradación del rendimiento.

### Ejercicios

* Explica la diferencia entre hilos (threads) y procesos.
* ¿Qué es una condición de carrera? Proporciona un ejemplo.
* ¿Cómo se pueden utilizar mutexes para prevenir condiciones de carrera en un programa multihilo?
* Implementa un programa en C utilizando Pthreads que sume los elementos de un arreglo grande dividiéndolo en segmentos, donde cada segmento es sumado por un hilo diferente.
    
    Instrucciones:
    * Divide el arreglo en partes iguales para cada hilo.
    * Cada hilo debe sumar su segmento del arreglo.
    * Los resultados parciales deben combinarse al final para obtener la suma total.

In [ ]:
## Tus respuestas

### OpenMP: Programación Paralela con Open Multi-Processing

OpenMP, acrónimo de Open Multi-Processing, es una API (Application Programming Interface) diseñada para facilitar la programación paralela en aplicaciones de alto rendimiento. OpenMP permite a los desarrolladores escribir programas que pueden ejecutarse en sistemas multiprocesador y multinúcleo, aprovechando la capacidad de procesamiento paralelo. Introducido en 1997, OpenMP es ampliamente utilizado en la industria y en la investigación científica para acelerar el rendimiento de las aplicaciones.

OpenMP facilita  el paralismo de datos con directivas que permiten distribuir bucles entre varios hilos y soporta el paralelismo de tareas mediante la creación de secciones paralelas.

OpenMP sigue el modelo de memoria compartida, donde todos los hilos tienen acceso al mismo espacio de memoria. Esto permite una comunicación eficiente entre los hilos pero también requiere una sincronización adecuada para evitar condiciones de carrera.

### API de OpenMP

OpenMP proporciona un conjunto de directivas, rutinas de biblioteca y variables de entorno para la programación paralela. Las directivas de OpenMP se utilizan para especificar regiones de código que deben ejecutarse en paralelo.

**Directiva #pragma omp parallel:**

Esta directiva crea un equipo de hilos que ejecutan el bloque de código asociado en paralelo.

In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    #pragma omp parallel
    {
        int id = omp_get_thread_num();
        printf("Hello from thread %d\n", id);
    }
    return 0;
}


**Directiva #pragma omp for:**

Se utiliza para paralelizar bucles for. Los iteradores del bucle se dividen entre los hilos disponibles.
Ejemplo

In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    int i, n = 10;
    int a[n];

    #pragma omp parallel for
    for (i = 0; i < n; i++) {
        a[i] = i * i;
        printf("Thread %d computes a[%d] = %d\n", omp_get_thread_num(), i, a[i]);
    }

    return 0;
}


**Directiva #pragma omp sections:**

Permite dividir el trabajo en secciones independientes que se ejecutan en paralelo.

In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    #pragma omp parallel sections
    {
        #pragma omp section
        {
            printf("Section 1 executed by thread %d\n", omp_get_thread_num());
        }
        #pragma omp section
        {
            printf("Section 2 executed by thread %d\n", omp_get_thread_num());
        }
    }

    return 0;
}


**Variables privadas y compartidas:**

- Privadas: Cada hilo tiene su propia copia de la variable.
- Compartidas: Todas las variables globales y estáticas son compartidas por defecto entre los hilos.

In [ ]:
int x;
#pragma omp parallel private(x)
{
    x = omp_get_thread_num();
    printf("Thread %d has private x = %d\n", omp_get_thread_num(), x);
}


**Ejemplo**

In [ ]:
// openmp_example.c
#include <omp.h>
#include <stdio.h>

int main() {
    #pragma omp parallel
    {
        int id = omp_get_thread_num();
        printf("Hello from thread %d\n", id);
    }
    return 0;
}


Para compilar y ejecutar:

gcc -o openmp_example openmp_example.c -fopenmp

./openmp_example


#### Sincronización en OpenMP

La sincronización es crucial para evitar conflictos y asegurar la coherencia de los datos en programas paralelos. OpenMP proporciona varios mecanismos de sincronización:

**Directiva #pragma omp critical:**

Define una sección crítica del código que solo puede ser ejecutada por un hilo a la vez.

In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    int sum = 0;
    #pragma omp parallel for
    for (int i = 0; i < 10; i++) {
        #pragma omp critical
        sum += i;
    }
    printf("Sum = %d\n", sum);
    return 0;
}


**Directiva #pragma omp barrier:**

Hace que todos los hilos esperen en este punto hasta que todos los hilos del equipo hayan llegado a la barrera.

#include <omp.h>
#include <stdio.h>

int main() {
    #pragma omp parallel
    {
        int id = omp_get_thread_num();
        printf("Before barrier - Thread %d\n", id);
        #pragma omp barrier
        printf("After barrier - Thread %d\n", id);
    }
    return 0;
}


**Directiva #pragma omp atomic:**

Garantiza que una operación de actualización de memoria se realice de forma atómica.

In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    int count = 0;
    #pragma omp parallel for
    for (int i = 0; i < 1000; i++) {
        #pragma omp atomic
        count++;
    }
    printf("Count = %d\n", count);
    return 0;
}


**Consideraciones y buenas prácticas**

- Balanceo de Carga: Es importante asegurar que el trabajo se distribuya equitativamente entre los hilos para evitar el sobrecarga de algunos hilos mientras otros están inactivos.
- Minimización de sincronización: Aunque la sincronización es necesaria, debe ser minimizada para reducir el overhead y mejorar la eficiencia.
- Debugging: La depuración de programas paralelos puede ser más compleja que en programas secuenciales. Herramientas especializadas y técnicas de logging pueden ser útiles.

### Ejercicios

- Describe cómo funciona la directiva #pragma omp parallel y su uso en un programa.
- ¿Qué es la directiva #pragma omp for y cómo se utiliza para paralelizar bucles?
- Explica la diferencia entre las variables privadas y compartidas en el contexto de OpenMP
- Escribe un programa en C que realice la multiplicación de dos matrices utilizando OpenMP para paralelizar el cálculo.

    Instrucciones:
    * Inicializa dos matrices de tamaño NxN con valores aleatorios.
    * Utiliza #pragma omp parallel for para paralelizar el bucle de multiplicación de matrices.
    * Asegúrate de manejar correctamente las variables compartidas y privadas.

In [ ]:
## Tus respuestas

### MPI: Interfaz de paso de mensajes (Message Passing Interface)


MPI, acrónimo de Message Passing Interface, es un estándar para la programación paralela que permite la comunicación y coordinación entre procesos que se ejecutan en un entorno distribuido. MPI es ampliamente utilizado en aplicaciones de computación de alto rendimiento (HPC), tales como simulaciones científicas, análisis de grandes datos y procesamiento de imágenes. Introducido en 1994, MPI proporciona una plataforma robusta y flexible para desarrollar aplicaciones que pueden escalar desde unas pocas máquinas hasta miles de nodos en supercomputadoras.

MPI se basa en el modelo de memoria distribuida.

En MPI, los programas están compuestos por múltiples procesos independientes que pueden ejecutarse en diferentes nodos de una red.
La comunicación entre procesos se realiza mediante el envío y recepción de mensajes. MPI proporciona una variedad de funciones para la comunicación punto a punto y colectiva.


**API de MPI**

MPI define una serie de funciones que permiten la inicialización, comunicación y finalización de programas paralelos. Algunas de las funciones más importantes incluyen:

MPI_Init y MPI_Finalize:

Prototipos: `int MPI_Init(int *argc, char ***argv);` y `int MPI_Finalize(void);`

Descripción: `MPI_Init` inicializa el entorno MPI, y `MPI_Finalize` lo finaliza.


In [ ]:
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    printf("Hello, MPI World!\n");
    MPI_Finalize();
    return 0;
}


MPI_Comm_size y MPI_Comm_rank:

Prototipos: `int MPI_Comm_size(MPI_Comm comm, int *size);` y `int MPI_Comm_rank(MPI_Comm comm, int *rank);`

Descripción: MPI_Comm_size obtiene el número de procesos en el comunicador, y MPI_Comm_rank obtiene el identificador (rango) del proceso.

In [ ]:
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_size, world_rank;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    printf("Hello from process %d of %d\n", world_rank, world_size);

    MPI_Finalize();
    return 0;
}


MPI_Send y MPI_Recv:

Prototipos: `int MPI_Send(const void *buf, int count, MPI_Datatype datatype, int dest, int tag, MPI_Comm comm);` y `int MPI_Recv(void *buf, int count, MPI_Datatype datatype, int source, int tag, MPI_Comm comm, MPI_Status *status);`

Descripción: MPI_Send envía un mensaje a un proceso destino, y MPI_Recv recibe un mensaje de un proceso fuente.

In [ ]:
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    if (world_rank == 0) {
        int number = 42;
        MPI_Send(&number, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
    } else if (world_rank == 1) {
        int number;
        MPI_Recv(&number, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received number %d from process 0\n", number);
    }

    MPI_Finalize();
    return 0;
}


MPI_Bcast:

Prototipo: `int MPI_Bcast(void *buffer, int count, MPI_Datatype datatype, int root, MPI_Comm comm);`

Descripción: Difunde un mensaje desde el proceso raíz a todos los demás procesos en el comunicador.

In [ ]:
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int data = 0;
    if (world_rank == 0) {
        data = 100;
    }

    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);
    printf("Process %d received data %d\n", world_rank, data);

    MPI_Finalize();
    return 0;
}


MPI_Reduce:

Prototipo: `int MPI_Reduce(const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op, int root, MPI_Comm comm);`

Descripción: Realiza una operación de reducción (como suma, máximo, etc.) sobre los datos distribuidos y recoge el resultado en el proceso raíz.

In [ ]:
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank, world_size;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    int local_data = world_rank;
    int global_sum;

    MPI_Reduce(&local_data, &global_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (world_rank == 0) {
        printf("Sum of all ranks is %d\n", global_sum);
    }

    MPI_Finalize();
    return 0;
}


MPI proporciona varias funciones para sincronizar procesos y garantizar la coherencia de los datos:

MPI_Barrier:

Prototipo: `int MPI_Barrier(MPI_Comm comm);`

Descripción: Bloquea hasta que todos los procesos en el comunicador han llegado a este punto.

In [ ]:
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    MPI_Barrier(MPI_COMM_WORLD);

    printf("All processes reached the barrier.\n");

    MPI_Finalize();
    return 0;
}


**Ejemplo**

In [ ]:
// mpi_example.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int world_size;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    printf("Hello from rank %d out of %d processors\n", world_rank, world_size);

    MPI_Finalize();
    return 0;
}


Para compilar y ejecutar:

mpicc -o mpi_example mpi_example.c

mpirun -np 4 ./mpi_example


**Ejemplo práctico de MPI**

Un ejemplo sencillo que ilustra el uso de MPI para calcular la suma de elementos en un arreglo distribuido:

In [ ]:
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_size, world_rank;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int n = 100;
    int *data = NULL;
    int local_n = n / world_size;
    int *local_data = (int*)malloc(local_n * sizeof(int));

    if (world_rank == 0) {
        data = (int*)malloc(n * sizeof(int));
        for (int i = 0; i < n; i++) {
            data[i] = i + 1;
        }
    }

    MPI_Scatter(data, local_n, MPI_INT, local_data, local_n, MPI_INT, 0, MPI_COMM_WORLD);

    int local_sum = 0;
    for (int i = 0; i < local_n; i++) {
        local_sum += local_data[i];
    }

    int global_sum;
    MPI_Reduce(&local_sum, &global_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (world_rank == 0) {
        printf("Total sum = %d\n", global_sum);
        free(data);
    }

    free(local_data);
    MPI_Finalize();
    return 0;
}


**Consideraciones y buenas prácticas**

- Escalabilidad: Diseña tu programa para escalar eficientemente a medida que aumentas el número de procesos.
- Evitación de deadlocks: Asegúrate de que las operaciones de comunicación no provoquen bloqueos.
- Optimización de comunicación: Minimiza la comunicación innecesaria entre procesos para mejorar el rendimiento.

### Ejercicios

- Explica el modelo de comunicación de MPI y cómo difiere de la comunicación basada en hilos.
- Describe las funciones básicas de MPI para enviar y recibir mensajes (MPI_Send y MPI_Recv).
- ¿Qué es MPI_Barrier y cuándo se debe utilizar?
- Implementa un programa en C utilizando MPI donde el proceso de rank 0 envíe un mensaje a todos los demás procesos utilizando MPI_Bcast.

    Instrucciones:
    * Inicializa MPI y determina el rank del proceso.
    * Si el proceso tiene rank 0, inicializa un mensaje y utiliza MPI_Bcast para enviarlo a todos los demás procesos.
    * Los otros procesos deben recibir el mensaje y mostrarlo en la consola.

In [ ]:
## Tus respuestas